In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt

In [0]:
import random as rand
data = np.zeros((60000,168))
for i in range(60000):
    for j in range(168):
        if (i<3156):
            data[i][j] = rand.randint(70,100)
        else:
            data[i][j] = rand.randint(100,200)

In [0]:
columns = []
for i in range(7):
  for j in range(24):
    columns.append(str(i+6)+'/12/2019'+' '+str(j)+':00:00')
new_file_df = pd.DataFrame(data,columns=columns)
new_file_df['user_id']  = ['user_'+str(i+1) for i in range(60000)]
new_file_df.set_index('user_id',inplace=True)
new_file_df.head()

,6/12/2019 0:00:00,6/12/2019 1:00:00,6/12/2019 2:00:00,6/12/2019 3:00:00,6/12/2019 4:00:00,6/12/2019 5:00:00,6/12/2019 6:00:00,6/12/2019 7:00:00,6/12/2019 8:00:00,6/12/2019 9:00:00,6/12/2019 10:00:00,6/12/2019 11:00:00,6/12/2019 12:00:00,6/12/2019 13:00:00,6/12/2019 14:00:00,6/12/2019 15:00:00,6/12/2019 16:00:00,6/12/2019 17:00:00,6/12/2019 18:00:00,6/12/2019 19:00:00,6/12/2019 20:00:00,6/12/2019 21:00:00,6/12/2019 22:00:00,6/12/2019 23:00:00,7/12/2019 0:00:00,7/12/2019 1:00:00,7/12/2019 2:00:00,7/12/2019 3:00:00,7/12/2019 4:00:00,7/12/2019 5:00:00,7/12/2019 6:00:00,7/12/2019 7:00:00,7/12/2019 8:00:00,7/12/2019 9:00:00,7/12/2019 10:00:00,7/12/2019 11:00:00,7/12/2019 12:00:00,7/12/2019 13:00:00,7/12/2019 14:00:00,7/12/2019 15:00:00,...,11/12/2019 8:00:00,11/12/2019 9:00:00,11/12/2019 10:00:00,11/12/2019 11:00:00,11/12/2019 12:00:00,11/12/2019 13:00:00,11/12/2019 14:00:00,11/12/2019 15:00:00,11/12/2019 16:00:00,11/12/2019 17:00:00,11/12/2019 18:00:00,11/12/2019 19:00:00,11/12/2019 20:00:00,11/12/2019 21:00:00,11/12/2019 22:00:00,11/12/2019 23:00:00,12/12/2019 0:00:00,12/12/2019 1:00:00,12/12/2019 2:00:00,12/12/2019 3:00:00,12/12/2019 4:00:00,12/12/2019 5:00:00,12/12/2019 6:00:00,12/12/2019 7:00:00,12/12/2019 8:00:00,12/12/2019 9:00:00,12/12/2019 10:00:00,12/12/2019 11:00:00,12/12/2019 12:00:00,12/12/2019 13:00:00,12/12/2019 14:00:00,12/12/2019 15:00:00,12/12/2019 16:00:00,12/12/2019 17:00:00,12/12/2019 18:00:00,12/12/2019 19:00:00,12/12/2019 20:00:00,12/12/2019 21:00:00,12/12/2019 22:00:00,12/12/2019 23:00:00
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
user_1,82.0,83.0,79.0,77.0,80.0,82.0,94.0,86.0,97.0,87.0,86.0,87.0,93.0,70.0,71.0,71.0,81.0,83.0,70.0,87.0,83.0,89.0,77.0,91.0,100.0,96.0,94.0,81.0,91.0,74.0,90.0,82.0,73.0,97.0,91.0,71.0,81.0,71.0,100.0,99.0,...,98.0,95.0,98.0,94.0,86.0,82.0,75.0,82.0,84.0,87.0,93.0,91.0,94.0,84.0,83.0,71.0,70.0,86.0,72.0,75.0,99.0,98.0,84.0,73.0,86.0,70.0,70.0,85.0,76.0,72.0,82.0,70.0,90.0,97.0,79.0,94.0,90.0,92.0,71.0,99.0
user_2,72.0,73.0,81.0,89.0,79.0,76.0,72.0,88.0,92.0,81.0,100.0,96.0,98.0,80.0,70.0,90.0,93.0,82.0,80.0,95.0,98.0,91.0,71.0,74.0,98.0,82.0,91.0,89.0,91.0,76.0,88.0,90.0,73.0,99.0,89.0,100.0,88.0,76.0,96.0,85.0,...,95.0,100.0,74.0,79.0,96.0,75.0,88.0,93.0,80.0,84.0,75.0,95.0,70.0,94.0,91.0,85.0,77.0,75.0,95.0,100.0,90.0,98.0,85.0,81.0,87.0,99.0,97.0,98.0,96.0,82.0,100.0,85.0,80.0,85.0,92.0,90.0,86.0,98.0,75.0,95.0
user_3,85.0,88.0,71.0,88.0,97.0,98.0,89.0,70.0,98.0,95.0,74.0,88.0,98.0,100.0,95.0,89.0,74.0,90.0,75.0,73.0,88.0,96.0,84.0,90.0,83.0,70.0,74.0,80.0,97.0,99.0,77.0,84.0,99.0,79.0,92.0,81.0,80.0,100.0,79.0,74.0,...,86.0,81.0,99.0,88.0,83.0,71.0,72.0,78.0,83.0,90.0,82.0,79.0,76.0,78.0,80.0,96.0,92.0,72.0,99.0,76.0,83.0,75.0,76.0,84.0,77.0,71.0,94.0,74.0,91.0,92.0,93.0,77.0,84.0,82.0,98.0,70.0,89.0,89.0,84.0,71.0
user_4,71.0,79.0,95.0,82.0,92.0,70.0,92.0,88.0,84.0,91.0,99.0,76.0,86.0,80.0,96.0,87.0,94.0,83.0,89.0,91.0,75.0,96.0,73.0,93.0,82.0,100.0,92.0,72.0,73.0,77.0,90.0,70.0,96.0,98.0,71.0,82.0,74.0,73.0,81.0,98.0,...,75.0,94.0,80.0,85.0,78.0,84.0,95.0,100.0,88.0,81.0,83.0,89.0,99.0,96.0,95.0,88.0,84.0,76.0,92.0,85.0,81.0,93.0,75.0,75.0,98.0,87.0,73.0,70.0,81.0,91.0,99.0,73.0,86.0,96.0,72.0,90.0,89.0,85.0,95.0,82.0
user_5,88.0,83.0,74.0,91.0,93.0,96.0,78.0,97.0,83.0,92.0,83.0,100.0,95.0,99.0,95.0,73.0,86.0,98.0,86.0,100.0,100.0,87.0,70.0,98.0,93.0,71.0,95.0,84.0,77.0,99.0,95.0,83.0,92.0,75.0,92.0,75.0,79.0,92.0,92.0,76.0,...,94.0,96.0,93.0,87.0,100.0,71.0,93.0,82.0,75.0,72.0,95.0,79.0,79.0,85.0,86.0,76.0,85.0,92.0,99.0,74.0,87.0,78.0,90.0,76.0,82.0,71.0,71.0,92.0,71.0,90.0,80.0,73.0,98.0,91.0,70.0,95.0,70.0,72.0,87.0,83.0


In [0]:
from dateutil.rrule import *
from datetime import datetime, date

days = map(
    datetime.isoformat,
    rrule(HOURLY, dtstart=date(2019, 12, 6), until=date(2019, 12, 13))
)

In [0]:
days = list(days)[:-1]
new_file_df.columns = days

In [0]:
# new_file_df.reset_index(inplace=True)
new_file_df = new_file_df.transpose()
new_file_df.head()

user_id,user_1,user_2,user_3,user_4,user_5,user_6,user_7,user_8,user_9,user_10,user_11,user_12,user_13,user_14,user_15,user_16,user_17,user_18,user_19,user_20,user_21,user_22,user_23,user_24,user_25,user_26,user_27,user_28,user_29,user_30,user_31,user_32,user_33,user_34,user_35,user_36,user_37,user_38,user_39,user_40,...,user_59961,user_59962,user_59963,user_59964,user_59965,user_59966,user_59967,user_59968,user_59969,user_59970,user_59971,user_59972,user_59973,user_59974,user_59975,user_59976,user_59977,user_59978,user_59979,user_59980,user_59981,user_59982,user_59983,user_59984,user_59985,user_59986,user_59987,user_59988,user_59989,user_59990,user_59991,user_59992,user_59993,user_59994,user_59995,user_59996,user_59997,user_59998,user_59999,user_60000
2019-12-06T00:00:00,82.0,72.0,85.0,71.0,88.0,79.0,73.0,97.0,85.0,81.0,87.0,92.0,72.0,72.0,82.0,86.0,90.0,76.0,74.0,98.0,75.0,87.0,84.0,97.0,81.0,78.0,70.0,86.0,87.0,78.0,90.0,91.0,97.0,94.0,77.0,80.0,88.0,85.0,82.0,98.0,...,154.0,151.0,186.0,166.0,169.0,142.0,176.0,194.0,103.0,104.0,105.0,197.0,187.0,169.0,182.0,102.0,175.0,163.0,124.0,171.0,121.0,155.0,117.0,106.0,128.0,139.0,150.0,107.0,159.0,192.0,114.0,184.0,117.0,163.0,152.0,108.0,126.0,119.0,119.0,136.0
2019-12-06T01:00:00,83.0,73.0,88.0,79.0,83.0,99.0,89.0,79.0,92.0,87.0,73.0,92.0,91.0,94.0,94.0,75.0,70.0,82.0,71.0,75.0,95.0,74.0,76.0,94.0,79.0,85.0,99.0,95.0,91.0,76.0,92.0,100.0,75.0,99.0,89.0,78.0,74.0,84.0,76.0,88.0,...,148.0,186.0,135.0,179.0,147.0,136.0,114.0,124.0,172.0,158.0,110.0,128.0,158.0,168.0,105.0,115.0,194.0,198.0,111.0,158.0,115.0,127.0,139.0,131.0,108.0,190.0,152.0,101.0,140.0,100.0,175.0,187.0,185.0,113.0,159.0,169.0,118.0,187.0,194.0,146.0
2019-12-06T02:00:00,79.0,81.0,71.0,95.0,74.0,73.0,90.0,72.0,92.0,86.0,71.0,96.0,73.0,73.0,89.0,71.0,97.0,76.0,75.0,86.0,70.0,97.0,74.0,77.0,79.0,71.0,77.0,98.0,94.0,74.0,75.0,84.0,99.0,91.0,75.0,100.0,83.0,71.0,100.0,70.0,...,151.0,114.0,168.0,127.0,133.0,135.0,190.0,171.0,144.0,127.0,179.0,177.0,194.0,102.0,191.0,155.0,161.0,126.0,188.0,181.0,152.0,141.0,103.0,122.0,131.0,163.0,164.0,171.0,164.0,192.0,188.0,113.0,129.0,113.0,177.0,178.0,140.0,178.0,135.0,151.0
2019-12-06T03:00:00,77.0,89.0,88.0,82.0,91.0,73.0,78.0,94.0,92.0,82.0,83.0,93.0,79.0,73.0,85.0,79.0,83.0,72.0,93.0,78.0,94.0,84.0,75.0,100.0,89.0,72.0,86.0,95.0,82.0,94.0,87.0,72.0,74.0,85.0,98.0,90.0,81.0,76.0,70.0,79.0,...,124.0,158.0,136.0,164.0,119.0,185.0,145.0,198.0,131.0,152.0,138.0,165.0,144.0,142.0,163.0,175.0,155.0,135.0,147.0,189.0,118.0,111.0,131.0,167.0,170.0,119.0,133.0,121.0,135.0,189.0,120.0,125.0,182.0,163.0,165.0,116.0,157.0,118.0,160.0,103.0
2019-12-06T04:00:00,80.0,79.0,97.0,92.0,93.0,99.0,93.0,81.0,78.0,89.0,93.0,71.0,85.0,77.0,91.0,85.0,93.0,79.0,87.0,93.0,82.0,73.0,96.0,70.0,72.0,87.0,93.0,96.0,80.0,96.0,89.0,79.0,94.0,87.0,98.0,100.0,74.0,79.0,92.0,91.0,...,108.0,190.0,132.0,153.0,119.0,170.0,130.0,110.0,109.0,195.0,181.0,150.0,179.0,190.0,190.0,146.0,125.0,181.0,183.0,123.0,101.0,135.0,128.0,137.0,122.0,160.0,165.0,137.0,155.0,198.0,170.0,191.0,180.0,122.0,177.0,113.0,112.0,142.0,192.0,118.0


In [0]:
from matplotlib import pyplot
from statsmodels.tsa.ar_model import AR
from sklearn.metrics import mean_squared_error

user_id = input("Enter User_id of user:")
X = new_file_df.loc[:,str(user_id)]
train, test = X[1:len(X)-5], X[len(X)-5:]

model = AR(train, freq='H')
model_fit = model.fit()
print('Lag: %s' % model_fit.k_ar)
print('Coefficients: %s' % model_fit.params)
predictions = model_fit.predict(start=len(train), end=len(train)+len(test)-1, dynamic=False)
for i in range(len(predictions)):
    print('forescasted_units_at_future_hour_'+str(i+1)+'=%f' % (predictions[i]), "  expected=",test[i])
error = mean_squared_error(test, predictions)
print('Test MSE: %.3f' % error)



Enter User_id of user:user_6
Lag: 14
Coefficients: const         132.334997
L1.user_6      -0.069223
L2.user_6       0.061050
L3.user_6      -0.009487
L4.user_6      -0.158981
L5.user_6       0.027660
L6.user_6       0.059255
L7.user_6       0.029621
L8.user_6      -0.165263
L9.user_6      -0.030697
L10.user_6     -0.080672
L11.user_6     -0.064895
L12.user_6     -0.092537
L13.user_6     -0.004625
L14.user_6     -0.060746
dtype: float64
forescasted_units_at_future_hour_1=85.535699   expected= 77.0
forescasted_units_at_future_hour_2=84.671201   expected= 70.0
forescasted_units_at_future_hour_3=87.972524   expected= 99.0
forescasted_units_at_future_hour_4=85.477524   expected= 75.0
forescasted_units_at_future_hour_5=83.500300   expected= 93.0
Test MSE: 121.946


In [0]:
#Testing
model_fit.predict(start=14, end=24, dynamic=False)

2018-01-01 15:00:00    86.560545
2018-01-01 16:00:00    82.201330
2018-01-01 17:00:00    86.311714
2018-01-01 18:00:00    84.137248
2018-01-01 19:00:00    85.116612
2018-01-01 20:00:00    86.460244
2018-01-01 21:00:00    83.028271
2018-01-01 22:00:00    86.212170
2018-01-01 23:00:00    89.167632
2018-01-02 00:00:00    82.108134
2018-01-02 01:00:00    81.306520
Freq: H, dtype: float64

In [0]:
#Testing
current = datetime.now()
last = days[-1]
l = datetime.strptime(last,'%Y-%m-%dT%H:%M:%S')
gap = current - l
gap_hrs = gap.days*24
# print(gap_hrs, gap_hrs+10)
# pred = model_fit.predict(start= len(train), end= gap_hrs, dynamic=False)
model_fit.predict(start = 24*6, end = (24*7)+2, dynamic=False)

2019-12-12 01:00:00    79.223017
2019-12-12 02:00:00    81.252744
2019-12-12 03:00:00    84.154701
2019-12-12 04:00:00    85.789714
2019-12-12 05:00:00    83.520642
2019-12-12 06:00:00    85.983200
2019-12-12 07:00:00    80.798425
2019-12-12 08:00:00    82.582527
2019-12-12 09:00:00    83.662264
2019-12-12 10:00:00    89.459103
2019-12-12 11:00:00    83.476409
2019-12-12 12:00:00    84.376843
2019-12-12 13:00:00    82.499510
2019-12-12 14:00:00    86.535520
2019-12-12 15:00:00    83.099478
2019-12-12 16:00:00    81.995658
2019-12-12 17:00:00    87.764642
2019-12-12 18:00:00    84.611756
2019-12-12 19:00:00    85.535699
2019-12-12 20:00:00    84.671201
2019-12-12 21:00:00    87.972524
2019-12-12 22:00:00    85.477524
2019-12-12 23:00:00    83.500300
2019-12-13 00:00:00    84.437197
2019-12-13 01:00:00    83.684147
2019-12-13 02:00:00    84.517891
2019-12-13 03:00:00    84.821100
Freq: H, dtype: float64